In [ ]:
import os

def find_latest_cgmres_directory(directory):
    # List all files/directories within the specified directory
    directories = os.listdir(directory)

    # Filter directories that start with 'cgmres_debug_'
    cgmres_dirs = [
        d for d in directories
        if d.startswith("cgmres_debug_") and os.path.isdir(os.path.join(directory, d))
    ]

    # Raise an error if no relevant directories are found
    if not cgmres_dirs:
        raise FileNotFoundError("No cgmres debug directories found in the specified directory.")
    
    # Find the latest directory by extracting and comparing the timestamp part of the directory name
    latest_dir = max(cgmres_dirs, key=lambda d: int(d.split('_')[-1]))

    # Join the specified directory path with the latest directory name
    latest_dir_path = os.path.join(directory, latest_dir)

    return latest_dir_path

directory_path = "/home/kyoichi-sugahara/.ros/log"  # Set the directory path
latest_directory = find_latest_cgmres_directory(directory_path)
print(f"The latest cgmres debug directory is: {latest_directory}")



In [ ]:
from plotter import Plotter

plotter = Plotter(log_dir=latest_directory)
plotter.set_scales(2,5,2)
plotter.show()
# plotter.save()

In [ ]:
# import importlib
# import animator  
# importlib.reload(animator)
from animator import TrajectoryFollowing

anim = TrajectoryFollowing(
    log_dir=latest_directory
)
# anim.set_skip_frames(10)
anim.generate_animation()

In [ ]:
import os
import sys

relative_path = "../../mpc_lateral_controller/debug"
absolute_path = os.path.abspath(relative_path)

if os.path.exists(absolute_path):
    sys.path.append(absolute_path)
else:
    print(f"The path {absolute_path} does not exist.")

import importlib
import visualize_path

importlib.reload(visualize_path)
from visualize_path import plot_trajectory

# データディレクトリを指定する場合は、以下のコメントアウトを解除し、適切なパスを設定してください。
# data_directory = "/path/to/your/data/directory"
# plot_trajectory(data_directory)

# デフォルトの最新ディレクトリを使用する場合は、引数なしで呼び出します。
plot_trajectory()

# ウィンドウで表示したい場合は、以下のコードを実行してください。
# python3 src/autoware/universe/control/mpc_lateral_controller/debug/visualize_path.py